# Demonstration for using xhycom
**Author: Jun Sasaki, Coded on September 2, 2020, Updated on September 2, 2020**<br>
Demonstration for quick view of HYCOM data without annotation and decoration.

In [ ]:
import xhycom.utils as xh
from xhycom.utils import PlotConfig, Data, Plotter
import xarray as xr
from netCDF4 import Dataset
from IPython.display import HTML, display
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import Stamen
from cartopy.io.img_tiles import OSM
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Downloading and creating multi-time netcdf
- `xh.run_opendap(extent, time_start, time_end=None, dtime=3, tz='utc')`
- Netcdf file name is automatically generated as "hycom_" + datetime + ".nc" 
- This method should be used in general.

In [ ]:
#extent = (139.2, 140.2, 34.8, 35.8)  ### Mouth of Tokyo Bay
extent = (-82.0, -65.0, 28.0, 43.0)  ### Project Sandy
time_start, time_end, dtime = ('2012-10-28 12:00:00', '2012-10-29 00:00:00', 3)  ### dtime (int): time interval in hours

### Reading catalog from local
#xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime)
### Reading catalog from OPeNDAP
dirpath = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_53.X/data/'

xh.run_opendap(extent=extent, time_start=time_start, time_end=time_end, dtime=dtime, opendap=True, dirpath=dirpath)

# Loading netcdf time series files into xarray.Dataset

In [ ]:
ncfiles = 'hycom_lon-82.0lon-65.0_lat28.0lat43.0_2012*.nc'

with xr.open_mfdataset(ncfiles, parallel=True, concat_dim="time", data_vars='minimal', \
                       coords='minimal', compat='override') as dsts:
    print("Creating xarray.Dataset from " + str(ncfiles))
dsts

# 2D Plotting for plan view, vertical views, and timeseries

In [ ]:
### Specify plot type and variables
plot_type = 'plot_xy'
var = 'salinity' #'water_temp'
t=0
x=70
y=0
z=0
vmin=None
vmax=None
png_path = "./" + plot_type + ".png"
dpi=300

### Create Data instance and PlotConfig instance
da_var = Data(da=dsts[var])
cfg_sal = PlotConfig()

### Create Plotter instance and invoke its method of save or plot

if   plot_type == 'plot_xy':  ### xy plot (lon, lat)
    plot_xy=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y='lat', z=z, t=t)
    plot_xy.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
elif plot_type == 'plot_xz':  ### xz plot (lon, depth)
    plot_xz=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y=y, z='depth', t=0)
    plot_xz.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
elif plot_type == 'plot_yz':  ### yz plot (lat, depth)
    plot_yz=Plotter(plot_config=cfg_sal, data=da_var, x=x, y='lat', z='depth', t=t)
    plot_yz.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
elif plot_type == 'plot_tx':  ### tx plot (time, lon)
    plot_tx=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y=y, z=z, t='time')
    plot_tx.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
elif plot_type == 'plot_ty':  ### ty plot (time, lat)
    plot_ty=Plotter(plot_config=cfg_sal, data=da_var, x=x, y='lat', z=z, t='time')
    plot_ty.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
elif plot_type == 'plot_tz':  ### tz plot (time, depth)
    plot_tz=Plotter(plot_config=cfg_sal, data=da_var, x=x, y=y, z='depth', t='time')
    plot_tz.plot(vmin=vmin, vmax=vmax).save(png_path, dpi=dpi, bbox_inches='tight')
else:
    'ERROR: No such plot_type'

# Creating GIF animation
Only supported plot_type of `plot_xy`. It is recommended to prepare a specific manual code for animation without using class because cutomizing panels is often required.

In [ ]:
plot_type = 'plot_xy'
gif_file = "./" + plot_type + ".gif"
var = 'salinity' #'water_temp'
z=0
vmin = 34
vmax = 36
levels = None
### Create Data instance and PlotConfig instance
da_var = Data(da=dsts[var])
cfg_sal = PlotConfig()

if plot_type == 'plot_xy':  ### xy plot (lon, lat)
    plot_xy=Plotter(plot_config=cfg_sal, data=da_var, x='lon', y='lat', z=z, t=slice(None))
    ani = plot_xy.frame(vmin=vmin, vmax=vmax, levels=levels, \
                  cbar_kwargs={'shrink':0.8}).anim(gif_file, frames=None, \
                                                   interval=500, writer='pillow', dpi=200)

## Display GIF animation

In [ ]:
display(HTML(ani.to_jshtml()))

## Create and display MP4 animation
Instance of animation is converted to MP4 animation.

# Interactive time series plotting with hvPlot
- Using dask for large data handling
- See [Readng and writing files - xarray](http://xarray.pydata.org/en/stable/io.html)

## Plan view
`xarray.Dataset.hvplot.quadmesh(x, y, z, project, coastline, frame_height, cmap)`

In [ ]:
z = 'salinity'
project = ccrs.PlateCarree()
frame_height = 300
cmap = 'magma_r'

dsts.hvplot.quadmesh(x='lon', y='lat', z=z, project=project, coastline='10m', \
                     frame_height=frame_height, cmap=cmap).opts(bgcolor='lightgray')

## Vertical sectional views

In [ ]:
x = 'lon'
z = 'salinity'
frame_width  = 300
frame_height = 200
cmap = 'magma_r'

dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, \
                     cmap=cmap).opts(bgcolor='lightgray')

In [ ]:
x = 'lat'
z = 'salinity'
dsts.hvplot.quadmesh(x=x, y='depth', z=z, flip_yaxis=True, frame_height=frame_height, frame_width=frame_width, \
                     cmap=cmap).opts(bgcolor='lightgray')